<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [27]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [28]:
print(nlp.pipe_names)

for pipeline in nlp.pipe_names[3::] : nlp.disable_pipe(pipeline)

print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
['tok2vec', 'morphologizer', 'parser']


In [29]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [30]:
doc = nlp(s1)
formatted_tokens = " ".join([f"{token.text} [{token.pos_}, {token.dep_}]" for token in doc])
print(formatted_tokens)


San [PRON, nsubj] Francisco [PROPN, flat:name] envisage [VERB, ROOT] d' [ADP, case] interdire [NOUN, obl:arg] les [DET, det] robots [NOUN, obj] coursiers [ADJ, amod] sur [ADP, case] les [DET, det] trottoirs [NOUN, nmod]


**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [31]:
# Veuillez écrire votre code ici.
subtrees = [" ".join([t.text for t in token.subtree]) for token in filter(lambda t: t.dep_ in ["nsubj", "obj"], doc)]

print(subtrees)

['San Francisco', 'les robots coursiers sur les trottoirs']


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

In [32]:
# Dans la  colonne 8. Cette colonne spécifie le type de relation de dépendance qui lie le mot courant à son gouverneur. 
# Elle contient des étiquettes comme "nsubj" (sujet nominal), "obj" (objet), "det" (déterminant), "amod" (modificateur adjectival)
# comme décrit dans la documentation: https://universaldependencies.org/format.html


**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [33]:
from spacy.tokens import DocBin, Doc
import itertools
test_data = DocBin().from_disk("spacy_data/fr-ud-test.spacy")

generator = test_data.get_docs(nlp.vocab)
s2 = next(itertools.islice(next(itertools.islice(generator, 1, None)).sents, 6, None))

**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [34]:
from spacy import displacy
from IPython.display import HTML, display

def displayTree(doc):
    display(HTML('<span class="tex2jax_ignore">{}</span>'.format(displacy.render(doc, style="dep", jupyter=False))))


In [35]:
displayTree(s2.as_doc())

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [36]:
displayTree(nlp(s2.as_doc()))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

In [37]:
# Différences sur les 10 dépendances:
#       la dépendance "advmod" entre ans -> tard est remplacée par "obl:mod" entre plus -> ans
#       la dépendance "obl" entre tient -> ans est remplacée par la dépencance "advmod" entre tient -> tard

# Taux de correction: On a 2 erreurs dans la prédiction pour 11 dépendances => l'accuracy est de 9/11 = 81.8%

**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [38]:
from spacy.scorer import Scorer
from spacy.training import Example

In [39]:
scorer = Scorer()
print(f"{scorer.score([Example(nlp(s2.as_doc()), s2.as_doc())])['dep_uas']:.3f}")


0.818


## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [40]:
examples = []
for doc in test_data.get_docs(nlp.vocab):
    examples.append(Example(nlp(doc.text), doc.copy()))

# Évaluation
scores = scorer.score(examples)

print("\nRésultats de l'évaluation du dependency parser:")
print(f"UAS: {scores['dep_uas']:.3f}")
print(f"LAS: {scores['dep_las']:.3f}")

# Comparaison avec les scores de la documentation
print("\nComparaison avec les scores de la documentation:")
doc_uas = 88
doc_las = 84
print(f"Documentation UAS: {doc_uas:.1f}% | Évaluation UAS: {scores['dep_uas']*100:.1f}%")
print(f"Documentation LAS: {doc_las:.1f}% | Évaluation LAS: {scores['dep_las']*100:.1f}%")
print(f"Différence UAS: {scores['dep_uas']*100 - doc_uas:.1f} points de pourcentage")
print(f"Différence LAS: {scores['dep_las']*100 - doc_las:.1f} points de pourcentage")



Résultats de l'évaluation du dependency parser:
UAS: 0.740
LAS: 0.610

Comparaison avec les scores de la documentation:
Documentation UAS: 88.0% | Évaluation UAS: 74.0%
Documentation LAS: 84.0% | Évaluation LAS: 61.0%
Différence UAS: -14.0 points de pourcentage
Différence LAS: -23.0 points de pourcentage


**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [47]:
from tabulate import tabulate

det_scores = scores["dep_las_per_type"]
det_scores = sorted(det_scores.items(), key=lambda x: x[1]["f"], reverse=True)

print(
    tabulate(
        [
            (dep, f"{score['p']:.3f}", f"{score['r']:.3f}",
            f"{score['f']:.3f}") for dep, score in det_scores
        ],
        headers=["Dep", "Precision", "Recall", "F1"]
    )
)

Dep           Precision    Recall     F1
----------  -----------  --------  -----
det               0.933     0.758  0.837
cc                0.801     0.779  0.79
case              0.858     0.727  0.788
mark              0.74      0.775  0.757
aux:pass          0.636     0.925  0.754
nsubj:pass        0.667     0.84   0.743
cop               0.729     0.734  0.731
nsubj             0.773     0.673  0.72
root              0.652     0.774  0.708
amod              0.665     0.736  0.699
nummod            0.764     0.644  0.699
advmod            0.699     0.683  0.691
obj               0.689     0.675  0.682
flat:name         0.673     0.605  0.637
nmod              0.583     0.595  0.589
xcomp             0.5       0.613  0.551
acl:relcl         0.519     0.547  0.532
acl               0.432     0.405  0.418
iobj              0.318     0.56   0.406
conj              0.395     0.375  0.384
advcl             0.392     0.373  0.382
ccomp             0.455     0.317  0.374
fixed             

## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [ ]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
!python -m spacy train config.cfg \
  --output ./myDEPparser1 \
  --paths.train ../Labo2/spacy_data/fr-ud-train.spacy \
  --paths.dev ../Labo2/spacy_data/fr-ud-dev.spacy \
  --verbose

In [ ]:
# Veuillez indiquer ici le nombre d'époques final.


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [ ]:
# Veuillez écrire votre code ici.


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [ ]:
# Veuillez écrire votre code ici.


**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.